In [ ]:
#limpio la memoria
rm(list=ls())
gc()

#cargo librerias
library("data.table")
library("ROCR")
library("rpart")
library("rpart.plot")
library("caret")

In [ ]:

#Parametros entrada

#kcarpeta_datasets    <- "../input/laboratorio-de-implementacion-i-2021/"   #KAGGLE
kcarpeta_datasets    <- "~/buckets/b1/datasets/"                          #VM o Ubuntu

#Archivo con datos etiquetados para entrenamiento
karchivo_entrada      <-  paste0(kcarpeta_datasets, "paquete_premium_202011.csv")

#Archivo con datos sin etiquetar para generar la predicción
karchivo_predecir      <-  paste0(kcarpeta_datasets, "paquete_premium_202101.csv")

#Formato para submit en Kaggle
#karchivo_score      <-  "../input/uamds2020ldi1f1/Sample_201910_Fase_I.txt"

#Separador de campos en archivos
kcampos_separador     <-  "\t"

#Campo que identifica las muestras
kcampo_id             <-  "numero_de_cliente"

#Campo que contiene la clase a estimar
kclase_nomcampo       <-  "clase_ternaria"

#Valor de interés
kclase_valor_positivo <-  "BAJA+2"

#Campos a borrar para el entrenamiento
kcampos_a_borrar      <-  c(kcampo_id,kclase_nomcampo,"foto_mes")

#Campo que contendrá a la variable objetivo generada
kobjetivo             <-  "clase"

#Identificación del modelo
kmodelo               <-  "02-RPART"

#Ganancia por TP
kTPGain               <-  48750

#Pérdida por FP
kFPGain               <-  -1250

kSeed <- 0


#Establezco semilla aleatoria
set.seed(kSeed)



In [ ]:
#cargo los datos
dataset <- fread(karchivo_entrada)

#Para hacer pruebas rapidas puedo reducir el dataset a una fraccion
#subsample <- sample(1:nrow(dataset), .1 * nrow(dataset))
#dataset <- dataset[subsample,]

In [ ]:
#Genero la clase
dataset[,(kobjetivo) := as.integer(dataset[,..kclase_nomcampo] == kclase_valor_positivo)]

#Limpio campos a borrar
dataset[ ,  (kcampos_a_borrar) := NULL    ] 

#Completo missings
dataset[is.na(dataset)] <- 0

In [ ]:
# generacion del modelo
formula  <-  formula(paste(kobjetivo, "~ ."))

In [ ]:
train_rows <- createDataPartition(dataset$clase, p = .66, list = FALSE)

#train_rows <- sample(1:nrow(dataset), .66 * nrow(dataset))
train <- dataset[train_rows,]
test <- dataset[-train_rows,]

In [ ]:
#Genero el modelo
t0       <-  Sys.time()
modelo   <-  rpart(formula,   data = train, maxdepth = 2,  cp=0,   xval=0)
t1       <-  Sys.time()

tcorrida <-  as.numeric( t1 - t0, units = "secs")
print( tcorrida)

In [ ]:
#Evalúo sobre test
predicted <- predict(modelo, test,) # predicted values

In [ ]:
#Detalles del modelo
modelo

In [ ]:
#Arbol generado
options(repr.plot.width=15, repr.plot.height=15)
prp(modelo, extra=101, digits=5, branch=1, type=4, varlen=0, faclen=0, tweak=1.3)

In [ ]:
#Calculo ganancia sobre dataset de pruebas
test$score <- predicted
test$Predicted <- test$score > 0.025

#Selecciono primeros N
#seleccionar = 1400
#top_n = order(-test$score)[1:seleccionar]
#test[top_n,"Predicted"] <- 1

#calculo la ganancia de incentivar cada caso
test[  , ganancia:= kFPGain ] #-1250
test[ clase==1,   ganancia:= kTPGain] #48750


#Determino ganancia
print(paste("La Ganancia es:", 3*sum(test$Predicted*test$ganancia)))

In [ ]:
#fwrite( test[,c('score','clase')], file="./salida_rpart_test.csv", sep="," )

In [ ]:
#Calculo las metricas
pred <- prediction(test$score, test$clase)

#Calculo las metricas
perf <- performance(pred, measure = "tpr", x.measure = "fpr")

#Grafico la ROC
plot(perf, col=rainbow(10))

#Determino AUC
performance( pred,"auc")@y.values

In [ ]:
#Genero el modelo sobre dataset completo

t0       <-  Sys.time()
modelo   <-  rpart(formula,   data = dataset, maxdepth = 2,  cp=0,   xval=0)
t1       <-  Sys.time()

tcorrida <-  as.numeric( t1 - t0, units = "secs")
print( tcorrida)

#Arbol generado
options(repr.plot.width=15, repr.plot.height=15)
prp(modelo, extra=101, digits=5, branch=1, type=4, varlen=0, faclen=0, tweak=1.3)

In [ ]:
#Leo datos a estimar
dataset_pred <- fread(karchivo_predecir)


#Genero la clase
#dataset_pred[,(kobjetivo) := as.integer(dataset[,..kclase_nomcampo] == kclase_valor_positivo)]

#Limpio campos a borrar
#dataset_pred[ ,  (kcampos_a_borrar) := NULL    ] 

#campo fold por consistencia
dataset_pred[ ,  fold := 0    ] 

prediccion  <- predict( modelo, dataset_pred ) #aplico el modelo

In [ ]:
dataset_pred[ , prob_baja := prediccion]
dataset_pred[ , Predicted  := as.numeric(prob_baja > 0.025) ]

entrega  <- dataset_pred[   , list(numero_de_cliente, Predicted) ] #genero la salida

#genero el archivo para Kaggle
fwrite( entrega, file="./salida_rpart.csv", sep="," )